In [1]:
import torch
from transformers import MarianMTModel, MarianTokenizer
import matplotlib.pyplot as plt
import numpy as np

# Загружаем модель и токенизатор
model_name = 'Helsinki-NLP/opus-mt-en-ar'
model = MarianMTModel.from_pretrained(model_name, output_attentions=True)
tokenizer = MarianTokenizer.from_pretrained(model_name)

def translate_with_attention(text: str):
    # Токенизируем текст
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    # Выполняем перевод
    outputs = model.generate(**inputs, return_dict_in_generate=True)
    translated_tokens = outputs.sequences

    # Выполняем проход вперед, чтобы получить веса внимания
    encoder_outputs = model.get_encoder()(inputs.input_ids, attention_mask=inputs.attention_mask, output_attentions=True)
    decoder_input_ids = model.prepare_decoder_input_ids_from_labels(labels=translated_tokens)
    decoder_outputs = model.get_decoder()(decoder_input_ids, encoder_hidden_states=encoder_outputs.last_hidden_state, output_attentions=True)

    attentions = decoder_outputs.attentions

    # Декодируем переведенный текст
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

    return translated_text, attentions, inputs.input_ids, translated_tokens

# Пример использования
english_text = "Hello, how are you?"
translated_text, attentions, input_ids, translated_tokens = translate_with_attention(english_text)
print(f"Original text: {english_text}")
print(f"Translated text: {translated_text}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:818: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_attentions` is. When `return_dict_in_generate` is not `True`, `output_attentions` is ignored.
  warnings.warn(


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

...

source.spm:   0%|          | 0.00/801k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Original text: Hello, how are you?
Translated text: مرحباً، كيف حالك؟
